In [1]:
import os
import sys

os.chdir('../..')
import torch
import numpy as np
from tqdm import tqdm
from rdkit import Chem
from rdkit.Chem.Descriptors import qed, MolWt
from rdkit.Chem.Draw import MolsToGridImage

from src.tacogfn.utils import sascore
from src.tacogfn.eval import docking
from src.tacogfn.utils import molecules
from src.tacogfn.tasks import pharmaco_frag

/home/tsa87/anaconda3/envs/tacogfn/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
import datamol as dm

data = dm.data.freesolv().sample(500).smiles.values
mols = [Chem.MolFromSmiles(smi) for smi in data]

In [3]:
model_state = torch.load('/home/tsa87/refactor-tacogfn/logs/20240121-new-zinc-mo-256-128/model_state_40000.pt')
trail = pharmaco_frag.PharmacophoreTrainer(model_state['cfg'])
trail.model.load_state_dict(model_state['models_state_dict'][0])
trail.model.eval()
test_idxs = trail.pharmaco_db.get_partition_idxs('test')

loaded 15108 ids for train
loaded 100 ids for test


Hyperparameters:

log_dir: ./logs/20240121-new-zinc-mo-256-128
logger: wandb
device: cuda
split_file: dataset/split_by_name.pt
pharmaco_db: misc/new_pharmacophores_db.lmdb
pocket_db: null
dock_pharmaco: dataset/affinity_prediction_pharmacophores/zinc_1000_dim_256
dock_proxy: model_weights/zinc_1000_dim_256.pth
avg_score: dataset/pocket_to_avg_zinc_vina_score.pt
info_only_dock_proxy: null
info_only_dock_pharmaco: null
seed: 0
validate_every: 1000
checkpoint_every: null
print_every: 100
start_at_step: 0
num_final_gen_steps: null
num_training_steps: 50000
num_workers: 8
hostname: d297022339ba
pickle_mp_messages: false
git_hash: cf09672
overwrite_existing_exp: true
algo:
  method: TB
  global_batch_size: 64
  max_len: 128
  max_nodes: 9
  max_edges: 128
  illegal_action_logreward: -75.0
  offline_ratio: 0.0
  valid_offline_ratio: 0.0
  train_random_action_prob: 0.01
  valid_random_action_prob: 0.0
  valid_sample_cond_info: true
  samplin

In [21]:
%%time
preds = trail.task.compute_flat_rewards(
    mols[:64],
    torch.tensor(test_idxs[:64])
)

CPU times: user 943 ms, sys: 0 ns, total: 943 ms
Wall time: 152 ms
